In [1]:
import math
#numpy is numerical python, which lets us do math!
import numpy as np
#pandas is for reading in and manipulating dataframe
import pandas as pd
#matplotlib is a plotting library the originated in matlab
from matplotlib import pyplot as plt
#seaborn wraps around matplotlib so that we can make pretty plots more easliy
import seaborn as sns
import missingno as msno
#this little line of code lets us see the plots in the notebook
%matplotlib inline
#this uses seaborn (sns) to set the style for all the plots
sns.set(context='poster', style='whitegrid', font_scale=1.4)
from os.path import join
from scipy.stats import ttest_ind, spearmanr, pearsonr, ttest_rel, kstest
import statsmodels as sm
from sklearn.linear_model import LinearRegression, Lasso
import fancyimpute as fi

#less important for plotting
from glob import glob
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
import pandas
pandas.__version__

'1.0.5'

In [3]:
def calculate_pvalues(df, correlation):
    from scipy.stats import pearsonr, pearsonr
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            if correlation == 'pearson':
                pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
            if correlation == 'spearman':
                pvalues[r][c] = round(spearmanr(df[r], df[c])[1], 4)
            else:
                raise ValueError('Invalid correlation method specified. Accpetable values are \'pearson\' and \'spearman\'.')
    return pvalues

In [4]:
#Li & Ji (2005) method for multiple comparisons corrections
#calculating number of effective comparisons M_eff
def liji_sidak_mc(data, alpha):
    import math
    import numpy as np
    
    mc_corrmat = data.corr()
    eigvals, eigvecs = np.linalg.eig(mc_corrmat)

    M_eff = 0
    for eigval in eigvals:
        if abs(eigval) >= 0:
            if abs(eigval) >= 1:
                M_eff += 1
            else:
                M_eff += abs(eigval) - math.floor(abs(eigval))
        else:
            M_eff += 0
    print('Number of effective comparisons: {0}'.format(M_eff))

    #and now applying M_eff to the Sidak procedure
    sidak_p = 1 - (1 - alpha)**(1/M_eff)
    if sidak_p < 0.00001:
        print('Critical value of {:.3f}'.format(alpha),'becomes {:2e} after corrections'.format(sidak_p))
    else:
        print('Critical value of {:.3f}'.format(alpha),'becomes {:.6f} after corrections'.format(sidak_p))
    return sidak_p, M_eff

In [5]:
def linreg_interxns(data, sessions, ivs, mods, task, condition, measures, parcellation, covariates, mc_alpha):
    index = pd.MultiIndex.from_product((sessions, measures))
    sig = []
    sig_mod = []
    brainXIQ_df = pd.DataFrame(index=index)
    for session in sessions:
        for measure in measures:
            for iv in ivs:
                if session == 'pre':
                    variables = ['{0}1'.format(iv)]
                if session == 'post':
                    variables = ['{0}2'.format(iv)]
                if session == 'delta':
                    variables = ['{0}2'.format(iv), 'delta{0}'.format(iv)]
                for var in variables:
                    exes = [var]
                    if mods:
                        for mod in mods:
                            data['{0}X{1}'.format(var, mod)] = data[var] * data[mod]
                            exes.append('{0}X{1}'.format(var, mod))
                    y = data['{0} {1} {2} {3} {4}'.format(session, condition, task, parcellation, measure)]
                    x = data[exes + covariates + ['{0} {1} {2} fd'.format(session, condition, task)]]
                    x = sm.add_constant(x)
                    reg1 = sm.OLS(endog=y, 
                                  exog=x, 
                                  missing='drop')
                    brain_on_iq = reg1.fit()
                    brainXIQ_df.at[(session, measure), 'F-statistic'] = brain_on_iq.fvalue
                    brainXIQ_df.at[(session, measure), 'Prob (F-statistic)'] = brain_on_iq.f_pvalue
                    brainXIQ_df.at[(session, measure), 'AIC'] = brain_on_iq.aic
                    brainXIQ_df.at[(session, measure), 'BIC'] = brain_on_iq.bic
                    for ex in exes:
                        brainXIQ_df.at[(session, measure), '{0}_pval'.format(ex)] = brain_on_iq.pvalues['{0}'.format(ex)]
                        brainXIQ_df.at[(session, measure), '{0}_beta'.format(ex)] = brain_on_iq.params['{0}'.format(ex)]
                    if brain_on_iq.f_pvalue < mc_alpha and brain_on_iq.pvalues['{0}'.format(var)] < 0.05:
                        print(brain_on_iq.summary())
                        #print(var, session, task, condition, measure, parcellation)
                        sig_mod.append(brain_on_iq)
                        sig.append('{0} {1} {2} {3} {4} {5}'.format(var, session, task, condition, measure, parcellation))
    return sig, sig_mod, brainXIQ_df

In [8]:
fig_dir = '/Users/kbottenh/Dropbox/Projects/physics-retrieval/figures'

In [11]:
data_dir = '/Users/kbottenh/Dropbox/Projects/physics-retrieval/data'
df = pd.read_csv(join(data_dir, 'rescored', 'physics_learning-nonbrain_OLS-imputed.csv'), 
                 index_col=0, header=0)

In [12]:
df.head()

,Age,RetrPhyAcc1,RetrPhyAcc2,FCIPhyAcc1,FCIPhyAcc2,VCI1,PRI1,WMI1,PSI1,FSIQ1,VCI2,PRI2,WMI2,PSI2,FSIQ2,Phy48Grade,F,Mod,StrtLvl
101,20.471233,0.708333,0.750000,0.333333,0.444444,102.0,81.0,100.0,100.0,94.0,108.000000,102.000000,95.000000,86.000000,99.000000,2.00000,1.0,0.0,3.0
102,21.202740,0.541667,0.833333,0.222222,0.555556,118.0,92.0,117.0,86.0,104.0,120.000000,100.000000,105.000000,122.000000,114.000000,3.00000,0.0,1.0,3.0
103,18.887671,0.750000,0.875000,0.444444,0.777778,130.0,92.0,89.0,74.0,98.0,130.000000,104.000000,102.000000,71.000000,104.000000,3.33000,0.0,0.0,3.0
104,21.342466,0.666667,0.708333,0.444444,0.555556,100.0,119.0,105.0,79.0,103.0,114.000000,123.000000,100.000000,79.000000,108.000000,4.00000,1.0,1.0,4.0
105,20.802740,0.723324,0.711568,0.332059,0.737803,116.0,119.0,117.0,74.0,110.0,109.696126,116.571411,95.649039,112.303741,111.635877,3.09011,0.0,0.0,2.0


In [13]:
#rsgt_df = pd.read_csv(join(data_dir, 'resting-state_graphtheory_shen+craddock_2019-05-29.csv'), 
#                      index_col=0, header=0)
#rsgt_df = pd.read_csv(join(data_dir, 'output', 'resting-state_graphtheory_shen+craddock_2019-11-20.csv'), 
#                      index_col=0, header=0)
head_size = pd.read_csv(join(data_dir, 'head-size_2019-05-29 15:19:53.287525.csv'), index_col=0, header=0)
head_size['normalized head size'] = (head_size['average_head_size']-np.mean(head_size['average_head_size']))/np.std(head_size['average_head_size'])

In [10]:
fd = pd.read_csv(join(data_dir, 'avg-fd-per-condition-per-run_2019-05-29.csv'), index_col=0, header=0)
fd['normalized fd'] = (fd['average fd']-np.mean(fd['average fd']))/np.std(fd['average fd'])
retr_fd = fd[fd['task'] == 'retr']
reas_fd = fd[fd['task'] == 'reas']
fci_fd = fd[fd['task'] == 'fci']

df_pivot = retr_fd[retr_fd['condition'] == 'high-level'].reset_index()
retr_phys_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
retr_phys_fd.rename({'pre': 'pre phys retr fd', 'post': 'post phys retr fd'}, axis=1, inplace=True)

df_pivot = retr_fd[retr_fd['condition'] == 'lower-level'].reset_index()
retr_genr_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
retr_genr_fd.rename({'pre': 'pre gen retr fd', 'post': 'post gen retr fd'}, axis=1, inplace=True)

#df_pivot = reas_fd[reas_fd['condition'] == 'high-level'].reset_index()
#reas_infr_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
#reas_infr_fd.rename({'pre': 'pre infr reas fd', 'post': 'post infr reas fd'}, axis=1, inplace=True)

#df_pivot = reas_fd[reas_fd['condition'] == 'lower-level'].reset_index()
#reas_ctrl_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
#reas_ctrl_fd.rename({'pre': 'pre ctrl reas fd', 'post': 'post ctrl reas fd'}, axis=1, inplace=True)

df_pivot = fci_fd[fci_fd['condition'] == 'high-level'].reset_index()
fci_phys_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
fci_phys_fd.rename({'pre': 'pre phys fci fd', 'post': 'post phys fci fd'}, axis=1, inplace=True)

df_pivot = fci_fd[fci_fd['condition'] == 'lower-level'].reset_index()
fci_ctrl_fd = df_pivot.pivot(index='subject', columns='session', values='average fd')
fci_ctrl_fd.rename({'pre': 'pre ctrl fci fd', 'post': 'post ctrl fci fd'}, axis=1, inplace=True)

In [11]:
#rest_fd = pd.read_csv(join(data_dir, 'avg-fd-per-run-rest_2019-05-31.csv'), index_col=0, header=0)
#rs_fd = rest_fd.reset_index().pivot(index='subject', columns='session', values='average fd')
#rs_fd.rename({'pre': 'pre rest fd', 'post': 'post rest fd'}, axis=1, inplace=True)

In [39]:
#fci_df = pd.read_csv(join(data_dir, 'physics-learning-fci_graphtheory_shen+craddock_2019-05-31.csv'), 
#                     index_col=0, header=0)

task_df = pd.read_csv(join(data_dir, 'output', 'physics-learning-tasks_graphtheory_shen+craddock.csv'), 
                     index_col=0, header=0)

task_df['raw assortativity'] = task_df['assortativity']
task_df['assortativity'] = (task_df['assortativity'] - np.mean(task_df['assortativity']))/np.std(task_df['assortativity'])
task_df['raw transitivity'] = task_df['transitivity']
task_df['transitivity'] = (task_df['transitivity'] - np.mean(task_df['transitivity']))/np.std(task_df['transitivity'])

fci_df = task_df[task_df['task'] == 'fci']
phys_fci = fci_df[fci_df['condition'] == 'high-level']
ctrl_fci = fci_df[fci_df['condition'] == 'lower-level']

#physics fci measures per session per parcellation
df_pivot = phys_fci[phys_fci['mask'] == 'shen2015'].reset_index()
shen_ast_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
shen_ast_phys_fci_df.rename({0: 'pre phys fci shen ast', 1: 'post phys fci shen ast'}, axis=1, inplace=True)

shen_trn_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
shen_trn_phys_fci_df.rename({0: 'pre phys fci shen trn', 1: 'post phys fci shen trn'}, axis=1, inplace=True)

df_pivot = phys_fci[phys_fci['mask'] == 'craddock2012'].reset_index()
crad_ast_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
crad_ast_phys_fci_df.rename({0: 'pre phys fci crad ast', 1: 'post phys fci crad ast'}, axis=1, inplace=True)

crad_trn_phys_fci_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
crad_trn_phys_fci_df.rename({0: 'pre phys fci crad trn', 1: 'post phys fci crad trn'}, axis=1, inplace=True)


#ctrl fci measures per session per parcellation
df_pivot = ctrl_fci[ctrl_fci['mask'] == 'shen2015'].reset_index()
shen_ast_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
shen_ast_ctrl_fci_df.rename({0: 'pre ctrl fci shen ast', 1: 'post ctrl fci shen ast'}, axis=1, inplace=True)

shen_trn_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
shen_trn_ctrl_fci_df.rename({0: 'pre ctrl fci shen trn', 1: 'post ctrl fci shen trn'}, axis=1, inplace=True)

df_pivot = ctrl_fci[ctrl_fci['mask'] == 'craddock2012'].reset_index()
crad_ast_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
crad_ast_ctrl_fci_df.rename({0: 'pre ctrl fci crad ast', 1: 'post ctrl fci crad ast'}, axis=1, inplace=True)

crad_trn_ctrl_fci_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
crad_trn_ctrl_fci_df.rename({0: 'pre ctrl fci crad trn', 1: 'post ctrl fci crad trn'}, axis=1, inplace=True)

In [40]:
#task_df = pd.read_csv(join(data_dir, 'physics-learning-tasks_graphtheory_shen+craddock_2019-05-30.csv'), index_col=0, header=0)
retr_df = task_df[task_df['task'] == 'retr']
retr_phys_df = retr_df[retr_df['condition'] == 'high-level']
genr_phys_df = retr_df[retr_df['condition'] == 'lower-level']

#physics retrieval measures per session per parcellation
df_pivot = retr_phys_df[retr_phys_df['mask'] == 'shen2015'].reset_index()
shen_ast_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
shen_ast_retr_phys_df.rename({0: 'pre phys retr shen ast', 1: 'post phys retr shen ast'}, axis=1, inplace=True)

shen_trn_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
shen_trn_retr_phys_df.rename({0: 'pre phys retr shen trn', 1: 'post phys retr shen trn'}, axis=1, inplace=True)

df_pivot = retr_phys_df[retr_phys_df['mask'] == 'craddock2012'].reset_index()
crad_ast_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
crad_ast_retr_phys_df.rename({0: 'pre phys retr crad ast', 1: 'post phys retr crad ast'}, axis=1, inplace=True)

crad_trn_retr_phys_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
crad_trn_retr_phys_df.rename({0: 'pre phys retr crad trn', 1: 'post phys retr crad trn'}, axis=1, inplace=True)

#general retrieval measures per session per parcellation
df_pivot = genr_phys_df[genr_phys_df['mask'] == 'shen2015'].reset_index()
shen_ast_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
shen_ast_retr_genr_df.rename({0: 'pre gen retr shen ast', 1: 'post gen retr shen ast'}, axis=1, inplace=True)

shen_trn_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
shen_trn_retr_genr_df.rename({0: 'pre gen retr shen trn', 1: 'post gen retr shen trn'}, axis=1, inplace=True)

df_pivot = genr_phys_df[genr_phys_df['mask'] == 'craddock2012'].reset_index()
crad_ast_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
crad_ast_retr_genr_df.rename({0: 'pre gen retr crad ast', 1: 'post gen retr crad ast'}, axis=1, inplace=True)

crad_trn_retr_genr_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
crad_trn_retr_genr_df.rename({0: 'pre gen retr crad trn', 1: 'post gen retr crad trn'}, axis=1, inplace=True)



reas_df = task_df[task_df['task'] == 'reas']
reas_infr_df = reas_df[reas_df['condition'] == 'high-level']
ctrl_infr_df = reas_df[reas_df['condition'] == 'lower-level']

#infrics reasieval measures per session per parcellation
df_pivot = reas_infr_df[reas_infr_df['mask'] == 'shen2015'].reset_index()
shen_ast_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
shen_ast_reas_infr_df.rename({0: 'pre infr reas shen ast', 1: 'post infr reas shen ast'}, axis=1, inplace=True)

shen_trn_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
shen_trn_reas_infr_df.rename({0: 'pre infr reas shen trn', 1: 'post infr reas shen trn'}, axis=1, inplace=True)

df_pivot = reas_infr_df[reas_infr_df['mask'] == 'craddock2012'].reset_index()
crad_ast_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
crad_ast_reas_infr_df.rename({0: 'pre infr reas crad ast', 1: 'post infr reas crad ast'}, axis=1, inplace=True)

crad_trn_reas_infr_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
crad_trn_reas_infr_df.rename({0: 'pre infr reas crad trn', 1: 'post infr reas crad trn'}, axis=1, inplace=True)

#ctrleral reasieval measures per session per parcellation
df_pivot = ctrl_infr_df[ctrl_infr_df['mask'] == 'shen2015'].reset_index()
shen_ast_reas_ctrl_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
shen_ast_reas_ctrl_df.rename({0: 'pre ctrl reas shen ast', 1: 'post ctrl reas shen ast'}, axis=1, inplace=True)

shen_trn_reas_ctrl_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
shen_trn_reas_ctrl_df.rename({0: 'pre ctrl reas shen trn', 1: 'post ctrl reas shen trn'}, axis=1, inplace=True)

df_pivot = ctrl_infr_df[ctrl_infr_df['mask'] == 'craddock2012'].reset_index()
crad_ast_reas_ctrl_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
crad_ast_reas_ctrl_df.rename({0: 'pre ctrl reas crad ast', 1: 'post ctrl reas crad ast'}, axis=1, inplace=True)

crad_trn_reas_ctrl_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
crad_trn_reas_ctrl_df.rename({0: 'pre ctrl reas crad trn', 1: 'post ctrl reas crad trn'}, axis=1, inplace=True)



rsgt_df['raw assortativity'] = rsgt_df['assortativity']
rsgt_df['assortativity'] = (rsgt_df['assortativity'] - np.mean(rsgt_df['assortativity']))/np.std(rsgt_df['assortativity'])
rsgt_df['raw transitivity'] = rsgt_df['transitivity']
rsgt_df['transitivity'] = (rsgt_df['transitivity'] - np.mean(rsgt_df['transitivity']))/np.std(rsgt_df['transitivity'])


df_pivot = rsgt_df[rsgt_df['mask'] == 'shen2015'].reset_index()
shen_ast_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
shen_ast_df.rename({0: 'pre rest shen ast', 1: 'post rest shen ast'}, axis=1, inplace=True)

shen_trn_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
shen_trn_df.rename({0: 'pre rest shen trn', 1: 'post rest shen trn'}, axis=1, inplace=True)

df_pivot = rsgt_df[rsgt_df['mask'] == 'craddock2012'].reset_index()
crad_ast_df = df_pivot.pivot(index='subject', columns='session', values='assortativity')
crad_ast_df.rename({0: 'pre rest crad ast', 1: 'post rest crad ast'}, axis=1, inplace=True)

crad_trn_df = df_pivot.pivot(index='subject', columns='session', values='transitivity')
crad_trn_df.rename({0: 'pre rest crad trn', 1: 'post rest crad trn'}, axis=1, inplace=True)

In [51]:
all_df = pd.concat((df, shen_ast_df, shen_trn_df,
                    crad_ast_df, crad_trn_df, 
                    head_size['normalized head size'],
                    shen_ast_retr_phys_df, shen_trn_retr_phys_df,
                    crad_ast_retr_phys_df, crad_trn_retr_phys_df, 
                    shen_ast_retr_genr_df, shen_trn_retr_genr_df, 
                    crad_ast_retr_genr_df, crad_trn_retr_genr_df,
                    shen_ast_phys_fci_df, shen_trn_phys_fci_df,
                    crad_ast_phys_fci_df, crad_trn_phys_fci_df,
                    shen_ast_ctrl_fci_df, shen_trn_ctrl_fci_df,
                    crad_ast_ctrl_fci_df, crad_trn_ctrl_fci_df, 
                    retr_phys_fd, retr_genr_fd, fci_phys_fd, fci_ctrl_fd, 
                    rs_fd, reas_ctrl_fd, reas_infr_fd, 
                    shen_ast_reas_infr_df, shen_trn_reas_infr_df, 
                    crad_ast_reas_infr_df, crad_trn_reas_infr_df, 
                    shen_ast_reas_ctrl_df, shen_trn_reas_ctrl_df, 
                    crad_ast_reas_ctrl_df, crad_trn_reas_ctrl_df), axis=1)

missing_df = pd.DataFrame(columns=all_df.keys(), index=all_df.index)

for key in missing_df.keys():
    missing_df[key] = all_df[key].isnull()

missing_df.replace({False: 0, True:1}, inplace=True)


In [52]:
index = pd.MultiIndex.from_product([missing_df.keys(), all_df.keys()])
mnar = pd.DataFrame(columns=['corr', 'pval'], index=index)

In [53]:
all_df.drop(['Sex', 'Class.Type'], axis=1, inplace=True)
all_df.dropna(inplace=True)

In [54]:
all_df['delta rest shen trn'] = all_df['post rest shen trn'] - all_df['pre rest shen trn']
all_df['delta rest shen ast'] = all_df['post rest shen ast'] - all_df['pre rest shen ast']
all_df['delta rest crad trn'] = all_df['post rest crad trn'] - all_df['pre rest crad trn']
all_df['delta rest crad ast'] = all_df['post rest crad ast'] - all_df['pre rest crad ast']

all_df['delta phys retr shen ast'] = all_df['post phys retr shen ast'] - all_df['pre phys retr shen ast']
all_df['delta phys retr shen trn'] = all_df['post phys retr shen trn'] - all_df['pre phys retr shen trn']
all_df['delta phys retr crad ast'] = all_df['post phys retr crad ast'] - all_df['pre phys retr crad ast']
all_df['delta phys retr crad trn'] = all_df['post phys retr crad trn'] - all_df['pre phys retr crad trn']

all_df['delta gen retr shen ast'] = all_df['post gen retr shen ast'] - all_df['pre gen retr shen ast']
all_df['delta gen retr shen trn'] = all_df['post gen retr shen trn'] - all_df['pre gen retr shen trn']
all_df['delta gen retr crad ast'] = all_df['post gen retr crad ast'] - all_df['pre gen retr crad ast']
all_df['delta gen retr crad trn'] = all_df['post gen retr crad trn'] - all_df['pre gen retr crad trn']

all_df['delta phys fci shen ast'] = all_df['post phys fci shen ast'] - all_df['pre phys fci shen ast']
all_df['delta phys fci shen trn'] = all_df['post phys fci shen trn'] - all_df['pre phys fci shen trn']
all_df['delta phys fci crad ast'] = all_df['post phys fci crad ast'] - all_df['pre phys fci crad ast']
all_df['delta phys fci crad trn'] = all_df['post phys fci crad trn'] - all_df['pre phys fci crad trn']

all_df['delta ctrl fci shen ast'] = all_df['post ctrl fci shen ast'] - all_df['pre ctrl fci shen ast']
all_df['delta ctrl fci shen trn'] = all_df['post ctrl fci shen trn'] - all_df['pre ctrl fci shen trn']
all_df['delta ctrl fci crad ast'] = all_df['post ctrl fci crad ast'] - all_df['pre ctrl fci crad ast']
all_df['delta ctrl fci crad trn'] = all_df['post ctrl fci crad trn'] - all_df['pre ctrl fci crad trn']

all_df['delta ctrl reas crad ast'] = all_df['post ctrl reas crad ast'] - all_df['pre ctrl reas crad ast']
all_df['delta ctrl reas crad trn'] = all_df['post ctrl reas crad trn'] - all_df['pre ctrl reas crad trn']
all_df['delta ctrl reas shen ast'] = all_df['post ctrl reas shen ast'] - all_df['pre ctrl reas shen ast']
all_df['delta ctrl reas shen trn'] = all_df['post ctrl reas shen ast'] - all_df['pre ctrl reas shen ast']

all_df['delta infr reas shen ast'] = all_df['post infr reas shen ast'] - all_df['pre infr reas shen ast']
all_df['delta infr reas shen trn'] = all_df['post infr reas shen trn'] - all_df['pre infr reas shen trn']
all_df['delta infr reas crad ast'] = all_df['post infr reas crad ast'] - all_df['pre infr reas crad ast']
all_df['delta infr reas crad trn'] = all_df['post infr reas crad trn'] - all_df['pre infr reas crad trn']

all_df['delta ctrl fci fd'] = (all_df['post ctrl fci fd'] + all_df['pre ctrl fci fd']) / 2
all_df['delta phys fci fd'] = (all_df['post phys fci fd'] + all_df['pre phys fci fd']) / 2
all_df['delta infr reas fd'] = (all_df['post infr reas fd'] + all_df['pre infr reas fd']) / 2
all_df['delta ctrl reas fd'] = (all_df['post ctrl reas fd'] + all_df['pre ctrl reas fd']) / 2
all_df['delta phys retr fd'] = (all_df['post phys retr fd'] + all_df['pre phys retr fd']) / 2
all_df['delta gen retr fd'] = (all_df['post gen retr fd'] + all_df['pre gen retr fd']) / 2
all_df['delta rest fd'] = (all_df['post rest fd'] + all_df['pre rest fd']) / 2

In [55]:
all_df_f = all_df[all_df['F'] == 1]
all_df_m = all_df[all_df['F'] == 0]

In [56]:
sexes = ['Male', 'Female', 'All']
sessions = ['pre', 'post', 'delta']

# For all students, all together, how is topology related to IQ?

In [57]:
#first things first, setting up our covariates
covariates = ['Age', 'Mod', 'Strt.Level', 'normalized head size', 'SexXClass', 'F']

In [58]:
iqs = ['VCI', 'WMI', 'PRI', 'PSI', 'FSIQ']

In [59]:
outcomes1 = ['pre phys fci shen ast', 'post phys fci shen ast',
            'pre phys fci shen trn', 'post phys fci shen trn',
            'pre ctrl fci shen ast', 'post ctrl fci shen ast', 
            'pre ctrl fci shen trn', 'post ctrl fci shen trn', 
            'delta phys fci shen trn',  'delta phys fci shen ast',
            'delta ctrl fci shen trn', 'delta ctrl fci shen ast']
outcomes2 = ['pre rest shen ast', 'post rest shen ast',  
            'pre rest shen trn', 'post rest shen trn', 
            'delta rest shen trn', 'delta rest shen ast']
outcomes3 = ['pre phys retr shen ast', 'post phys retr shen ast',
            'pre phys retr shen trn', 'post phys retr shen trn',
            'pre gen retr shen ast', 'post gen retr shen ast',
            'pre gen retr shen trn', 'post gen retr shen trn', 
            'delta phys retr shen trn', 'delta phys retr shen ast',
            'delta gen retr shen trn', 'delta gen retr shen ast']

all_out = outcomes1 + outcomes2 + outcomes3

shen_mc_alpha,_ = liji_sidak_mc(all_df[all_out], 0.05)
print('Shen', shen_mc_alpha)

Number of effective comparisons: 12.533597742652383
Critical value of 0.050 becomes 0.004084 after corrections
Shen 0.004084101029666942


In [60]:
outcomes1 = ['pre phys fci crad ast', 'post phys fci crad ast',
            'pre phys fci crad trn', 'post phys fci crad trn',
            'pre ctrl fci crad ast', 'post ctrl fci crad ast', 
            'pre ctrl fci crad trn', 'post ctrl fci crad trn', 
            'delta phys fci crad trn',  'delta phys fci crad ast',
            'delta ctrl fci crad trn', 'delta ctrl fci crad ast']
outcomes2 = ['pre rest crad ast', 'post rest crad ast',  
            'pre rest crad trn', 'post rest crad trn', 
            'delta rest crad trn', 'delta rest crad ast']
outcomes3 = ['pre phys retr crad ast', 'post phys retr crad ast',
            'pre phys retr crad trn', 'post phys retr crad trn',
            'pre gen retr crad ast', 'post gen retr crad ast',
            'pre gen retr crad trn', 'post gen retr crad trn', 
            'delta phys retr crad trn', 'delta phys retr crad ast',
            'delta gen retr crad trn', 'delta gen retr crad ast']

all_out = outcomes1 + outcomes2 + outcomes3

#
crad_mc_alpha,_ = liji_sidak_mc(all_df[all_out], 0.05)
print('Craddock', crad_mc_alpha)

Number of effective comparisons: 13.046864762298497
Critical value of 0.050 becomes 0.003924 after corrections
Craddock 0.003923747078351858


In [143]:
sessions = ['pre', 'post', 'delta']
ivs = ['VCI', 'WMI', 'PRI', 'PSI', 'FSIQ']
mods = ['SexXClass']
tasks = {'fci': ['phys', 'ctrl'], 
         'retr': ['phys', 'gen'], 
         'reas': ['infr', 'ctrl'], 
         'rest': []}
parcellations = ['crad', 'shen']
measures = ['ast', 'trn']
covariates = ['Age', 'Mod', 'Strt.Level', 'normalized head size', 'SexXClass', 'F']

In [144]:
sig = []
sig_mod = []
for task in tasks.keys():
        for condition in tasks[task]:
            for parcellation in parcellations:
                if parcellation == 'shen':
                    mc_alpha = shen_mc_alpha
                if parcellation == 'crad':
                    mc_alpha = crad_mc_alpha
                k = linreg_interxns(all_df, sessions, ivs, mods, task, condition, 
                                measures, parcellation, covariates, mc_alpha)
                sig.append(k[0])
                sig_mod.append(k[1])

                               OLS Regression Results                              
Dep. Variable:     post ctrl reas crad trn   R-squared:                       0.266
Model:                                 OLS   Adj. R-squared:                  0.189
Method:                      Least Squares   F-statistic:                     3.455
Date:                     Tue, 26 Nov 2019   Prob (F-statistic):            0.00112
Time:                             12:51:02   Log-Likelihood:                -89.576
No. Observations:                       96   AIC:                             199.2
Df Residuals:                           86   BIC:                             224.8
Df Model:                                9                                         
Covariance Type:                 nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [145]:
iqXclassxsex = sig_mod

In [123]:
iqXnone = sig_mod

In [150]:
r2_red = iqXnone[-1][0].rsquared
r2_ful = iqXclassxsex[-1][0].rsquared
p_red = iqXnone[-1][0].pvalues.shape[0]
p_ful = iqXclassxsex[-1][0].pvalues.shape[0]

In [151]:
((r2_ful - r2_red)/(1 - r2_ful))*((96-p_ful-p_red-1)/p_ful)

-0.1674499074612432

In [152]:
iqXclassxsex[-1][0].compare_f_test(iqXnone[-1][0])

(-1.8948279002193424, 1.0, 1.0)

## Rest and brain organization!

In [64]:
outcomes = ['pre rest crad ast', 'post rest crad ast', 
            'pre rest crad trn', 'post rest crad trn', 
            'delta rest crad trn', 'delta rest crad ast']

In [99]:
outcomes = ['pre rest shen ast', 'post rest shen ast',
            'pre rest shen trn', 'post rest shen trn', 
            'delta rest shen trn','delta rest shen ast']

In [100]:
mc_alpha,_ = liji_sidak_mc(all_df[outcomes], 0.05)

Number of effective comparisons: 3.635168491645377
Critical value of 0.050 becomes 0.014011 after corrections


In [101]:
index = pd.MultiIndex.from_product((sessions, iqs))
brainXIQ_df = pd.DataFrame(index=index, columns=['F-statistic', 'Prob (F-statistic)', 
                                                     'AIC', 'BIC', 'IQ_pval', 'IQXClass_pval'])

all_outcome_mc,_ = liji_sidak_mc(all_df[outcomes], 0.05)
for i in np.arange(0,len(outcomes)):
    for iq_meas in iqs:
        y = all_df[outcomes[i]]
        x = all_df[['{0}1'.format(iq_meas), 
                    #'{0}1XSex'.format(iq_meas), 
                    #'{0}1XClass'.format(iq_meas),
                    #'{0}1XClassXSex'.format(iq_meas), 
                    '{0} fd'.format(outcomes[i][:-9])
                   ] 
                   + covariates]

        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        if len(x.dropna()) == len(y.dropna()):
            linreg = Lasso()
            linreg.fit(x.dropna(),y.dropna())
        brain_on_pre_iq = reg1.fit()
        brainXIQ_df.at[('pre', iq_meas), 'F-statistic'] = brain_on_pre_iq.fvalue
        brainXIQ_df.at[('pre', iq_meas), 'Prob (F-statistic)'] = brain_on_pre_iq.f_pvalue
        brainXIQ_df.at[('pre', iq_meas), 'AIC'] = brain_on_pre_iq.aic
        brainXIQ_df.at[('pre', iq_meas), 'BIC'] = brain_on_pre_iq.bic
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQ_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[1]
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQXClass_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[2]
        if brain_on_pre_iq.f_pvalue < all_mc_alpha:
            print(brain_on_pre_iq.summary())
        #print('Course grade regressed on pre IQ, female students\n', grade_on_iq.summary())
        x = all_df[['{0}2'.format(iq_meas), 
                    #'{0}2XSex'.format(iq_meas), 
                    #'{0}2XClass'.format(iq_meas),
                    #'{0}2XClassXSex'.format(iq_meas), 
                    '{0} fd'.format(outcomes[i][:-9])] + covariates]

        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_post_iq = reg1.fit()
        brainXIQ_df.at[('post', iq_meas), 'F-statistic'] = brain_on_post_iq.fvalue
        brainXIQ_df.at[('post', iq_meas), 'Prob (F-statistic)'] = brain_on_post_iq.f_pvalue
        brainXIQ_df.at[('post', iq_meas), 'AIC'] = brain_on_post_iq.aic
        brainXIQ_df.at[('post', iq_meas), 'BIC'] = brain_on_post_iq.bic
        brainXIQ_df.at[('post', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[1]
        brainXIQ_df.at[('post', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[2]
        if brain_on_post_iq.f_pvalue < all_mc_alpha:
            print(brain_on_post_iq.summary())
        #print('Course grade regressed on post IQ, female students\n', grade_on_iq.summary())
        x = all_df[['delta{0}'.format(iq_meas), 
                    #'delta{0}XSex'.format(iq_meas), 
                    #'delta{0}XClass'.format(iq_meas), 
                    #'delta{0}XClassXSex'.format(iq_meas), 
                    '{0} fd'.format(outcomes[i][:-9])] + covariates]

        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_delta_iq = reg1.fit()
        brainXIQ_df.at[('delta', iq_meas), 'F-statistic'] = brain_on_delta_iq.fvalue
        brainXIQ_df.at[('delta', iq_meas), 'Prob (F-statistic)'] = brain_on_delta_iq.f_pvalue
        brainXIQ_df.at[('delta', iq_meas), 'AIC'] = brain_on_delta_iq.aic
        brainXIQ_df.at[('delta', iq_meas), 'BIC'] = brain_on_delta_iq.bic
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[1]
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[2]
        if brain_on_delta_iq.f_pvalue < all_mc_alpha:
            print(brain_on_delta_iq.summary())
    brainXIQ_df.to_csv(join(data_dir, 'rescored', 'iqX{0}_all.csv'.format(outcomes[i])))

Number of effective comparisons: 3.635168491645377
Critical value of 0.050 becomes 0.014011 after corrections


## Retrieval task brain organization and IQ!

In [68]:
outcomes = ['pre phys retr crad ast', 'post phys retr crad ast', 
            'pre phys retr crad trn', 'post phys retr crad trn',
            'pre gen retr crad ast', 'post gen retr crad ast',  
            'pre gen retr crad trn', 'post gen retr crad trn', 
            'delta phys retr crad trn', 'delta phys retr crad ast',
            'delta gen retr crad trn', 'delta gen retr crad ast']

In [69]:
outcomes = ['pre phys retr shen ast', 'post phys retr shen ast', 
            'pre phys retr shen trn', 'post phys retr shen trn',
            'pre gen retr shen ast', 'post gen retr shen ast',  
            'pre gen retr shen trn', 'post gen retr shen trn', 
            'delta phys retr shen trn', 'delta phys retr shen ast',
            'delta gen retr shen trn', 'delta gen retr shen ast']

In [70]:
index = pd.MultiIndex.from_product((sessions, iqs))
brainXIQ_df = pd.DataFrame(index=index, columns=['F-statistic', 'Prob (F-statistic)', 
                                                     'AIC', 'BIC', 'IQ_pval', 'IQXClass_pval'])
all_outcome_mc,_ = liji_sidak_mc(all_df[outcomes], 0.05)
for i in np.arange(0,len(outcomes)):
    for iq_meas in iqs:
        y = all_df[outcomes[i]]
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}1'.format(iq_meas), 
                        '{0}1XSex'.format(iq_meas), 
                        #'{0}1XClass'.format(iq_meas),
                        #'{0}1XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}1'.format(iq_meas), 
                        '{0}1XSex'.format(iq_meas), 
                        #'{0}1XClass'.format(iq_meas),
                        #'{0}1XClassXSex'.format(iq_meas)
                       ] + covariates]
            
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_pre_iq = reg1.fit()
        brainXIQ_df.at[('pre', iq_meas), 'F-statistic'] = brain_on_pre_iq.fvalue
        brainXIQ_df.at[('pre', iq_meas), 'Prob (F-statistic)'] = brain_on_pre_iq.f_pvalue
        brainXIQ_df.at[('pre', iq_meas), 'AIC'] = brain_on_pre_iq.aic
        brainXIQ_df.at[('pre', iq_meas), 'BIC'] = brain_on_pre_iq.bic
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQ_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[1]
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQXClass_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[2]
        if brain_on_pre_iq.f_pvalue < all_mc_alpha and brain_on_pre_iq.pvalues['{0}1'.format(iq_meas)] < 0.05:
            print(brain_on_pre_iq.summary())
        #print('Course grade regressed on pre IQ, female students\n', grade_on_iq.summary())
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        #'{0}2XClass'.format(iq_meas),
                        #'{0}2XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        #'{0}2XClass'.format(iq_meas),
                        #'{0}2XClassXSex'.format(iq_meas)
                       ] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_post_iq = reg1.fit()
        brainXIQ_df.at[('post', iq_meas), 'F-statistic'] = brain_on_post_iq.fvalue
        brainXIQ_df.at[('post', iq_meas), 'Prob (F-statistic)'] = brain_on_post_iq.f_pvalue
        brainXIQ_df.at[('post', iq_meas), 'AIC'] = brain_on_post_iq.aic
        brainXIQ_df.at[('post', iq_meas), 'BIC'] = brain_on_post_iq.bic
        brainXIQ_df.at[('post', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[1]
        brainXIQ_df.at[('post', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[2]
        if brain_on_post_iq.f_pvalue < all_mc_alpha and brain_on_post_iq.pvalues['{0}2'.format(iq_meas)] < 0.05:
            print(brain_on_post_iq.summary())
        #print('Course grade regressed on post IQ, female students\n', grade_on_iq.summary())
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['delta{0}'.format(iq_meas), 
                        'delta{0}XSex'.format(iq_meas), 
                        #'delta{0}XClass'.format(iq_meas), 
                        #'delta{0}XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['delta{0}'.format(iq_meas), 
                        'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas),
                        'delta{0}XClassXSex'.format(iq_meas)] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_delta_iq = reg1.fit()
        brainXIQ_df.at[('delta', iq_meas), 'F-statistic'] = brain_on_delta_iq.fvalue
        brainXIQ_df.at[('delta', iq_meas), 'Prob (F-statistic)'] = brain_on_delta_iq.f_pvalue
        brainXIQ_df.at[('delta', iq_meas), 'AIC'] = brain_on_delta_iq.aic
        brainXIQ_df.at[('delta', iq_meas), 'BIC'] = brain_on_delta_iq.bic
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[1]
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[2]
        if brain_on_delta_iq.f_pvalue < all_mc_alpha and brain_on_delta_iq.pvalues['delta{0}'.format(iq_meas)] < 0.05:
            print(brain_on_delta_iq.summary())
    brainXIQ_df.to_csv(join(data_dir, 'rescored', 'iqX{0}_all.csv'.format(outcomes[i])))

Number of effective comparisons: 5.1948813604449136
Critical value of 0.050 becomes 0.009825 after corrections


## Reasoning task and IQ!

In [71]:
outcomes = ['pre infr reas shen ast', 'post infr reas shen ast', 
            'pre infr reas shen trn', 'post infr reas shen trn',
            'pre ctrl reas shen ast', 'post ctrl reas shen ast', 
            'pre ctrl reas shen trn', 'post ctrl reas shen trn',
            'delta infr reas shen trn', 'delta infr reas shen ast',
            'delta ctrl reas shen trn', 'delta ctrl reas shen ast']

In [72]:
outcomes = ['pre infr reas crad ast', 'post infr reas crad ast', 
            'pre infr reas crad trn', 'post infr reas crad trn',
            'pre ctrl reas crad ast', 'post ctrl reas crad ast', 
            'pre ctrl reas crad trn', 'post ctrl reas crad trn',
            'delta infr reas crad trn', 'delta infr reas crad ast',
            'delta ctrl reas crad trn', 'delta ctrl reas crad ast']

In [73]:
index = pd.MultiIndex.from_product((sessions, iqs))
brainXIQ_df = pd.DataFrame(index=index, columns=['F-statistic', 'Prob (F-statistic)', 
                                                     'AIC', 'BIC', 'IQ_pval', 'IQXClass_pval'])
all_outcome_mc,_ = liji_sidak_mc(all_df[outcomes], 0.05)
for i in np.arange(0,len(outcomes)):
    for iq_meas in iqs:
        y = all_df[outcomes[i]]
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}1'.format(iq_meas), 
                        '{0}1XSex'.format(iq_meas), 
                        '{0}1XClass'.format(iq_meas),
                        '{0}1XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}1'.format(iq_meas), 
                        '{0}1XSex'.format(iq_meas), 
                        '{0}1XClass'.format(iq_meas),
                        '{0}1XClassXSex'.format(iq_meas)] + covariates]
            
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_pre_iq = reg1.fit()
        brainXIQ_df.at[('pre', iq_meas), 'F-statistic'] = brain_on_pre_iq.fvalue
        brainXIQ_df.at[('pre', iq_meas), 'Prob (F-statistic)'] = brain_on_pre_iq.f_pvalue
        brainXIQ_df.at[('pre', iq_meas), 'AIC'] = brain_on_pre_iq.aic
        brainXIQ_df.at[('pre', iq_meas), 'BIC'] = brain_on_pre_iq.bic
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQ_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[1]
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQXClass_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[2]
        if brain_on_pre_iq.f_pvalue < all_mc_alpha and brain_on_pre_iq.pvalues['{0}1'.format(iq_meas)] < 0.05:
            print(brain_on_pre_iq.summary())
        #print('Course grade regressed on pre IQ, female students\n', grade_on_iq.summary())
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        '{0}2XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}2'.format(iq_meas), 
                        '{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        '{0}2XClassXSex'.format(iq_meas)] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_post_iq = reg1.fit()
        brainXIQ_df.at[('post', iq_meas), 'F-statistic'] = brain_on_post_iq.fvalue
        brainXIQ_df.at[('post', iq_meas), 'Prob (F-statistic)'] = brain_on_post_iq.f_pvalue
        brainXIQ_df.at[('post', iq_meas), 'AIC'] = brain_on_post_iq.aic
        brainXIQ_df.at[('post', iq_meas), 'BIC'] = brain_on_post_iq.bic
        brainXIQ_df.at[('post', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[1]
        brainXIQ_df.at[('post', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[2]
        if brain_on_post_iq.f_pvalue < all_mc_alpha and brain_on_post_iq.pvalues['{0}2'.format(iq_meas)] < 0.05:
            print(brain_on_post_iq.summary())
        #print('Course grade regressed on post IQ, female students\n', grade_on_iq.summary())

        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['delta{0}'.format(iq_meas), 
                        'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas), 
                        'delta{0}XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['delta{0}'.format(iq_meas), 
                        'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas),
                        'delta{0}XClassXSex'.format(iq_meas)] + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_delta_iq = reg1.fit()
        brainXIQ_df.at[('delta', iq_meas), 'F-statistic'] = brain_on_delta_iq.fvalue
        brainXIQ_df.at[('delta', iq_meas), 'Prob (F-statistic)'] = brain_on_delta_iq.f_pvalue
        brainXIQ_df.at[('delta', iq_meas), 'AIC'] = brain_on_delta_iq.aic
        brainXIQ_df.at[('delta', iq_meas), 'BIC'] = brain_on_delta_iq.bic
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[1]
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[2]
        if brain_on_delta_iq.f_pvalue < all_mc_alpha and brain_on_delta_iq.pvalues['delta{0}'.format(iq_meas)] < 0.05:
            print(brain_on_delta_iq.summary())
    brainXIQ_df.to_csv(join(data_dir, 'rescored', 'iqX{0}_all.csv'.format(outcomes[i])))

Number of effective comparisons: 7.0801991032605205
Critical value of 0.050 becomes 0.007218 after corrections
                               OLS Regression Results                              
Dep. Variable:     post ctrl reas crad trn   R-squared:                       0.283
Model:                                 OLS   Adj. R-squared:                  0.189
Method:                      Least Squares   F-statistic:                     3.011
Date:                     Mon, 25 Nov 2019   Prob (F-statistic):            0.00200
Time:                             16:33:54   Log-Likelihood:                -88.434
No. Observations:                       96   AIC:                             200.9
Df Residuals:                           84   BIC:                             231.6
Df Model:                               11                                         
Covariance Type:                 nonrobust                                         
                           coef    std err       

                               OLS Regression Results                               
Dep. Variable:     delta infr reas crad trn   R-squared:                       0.300
Model:                                  OLS   Adj. R-squared:                  0.209
Method:                       Least Squares   F-statistic:                     3.280
Date:                      Mon, 25 Nov 2019   Prob (F-statistic):           0.000897
Time:                              16:33:54   Log-Likelihood:                -116.62
No. Observations:                        96   AIC:                             257.2
Df Residuals:                            84   BIC:                             288.0
Df Model:                                11                                         
Covariance Type:                  nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

## FCI Task

In [ ]:
outcomes = ['pre phys fci crad ast', 'post phys fci crad ast',
            'pre phys fci crad trn', 'post phys fci crad trn',
            'pre ctrl fci crad ast', 'post ctrl fci crad ast', 
            'pre ctrl fci crad trn', 'post ctrl fci crad trn', 
            'delta phys fci crad trn', 'delta phys fci crad ast',
            'delta ctrl fci crad trn', 'delta ctrl fci crad ast']

In [ ]:
outcomes = ['pre phys fci shen ast', 'post phys fci shen ast',
            'pre phys fci shen trn', 'post phys fci shen trn',
            'delta phys fci shen trn', 'delta phys fci shen ast']

In [ ]:
outcomes = ['pre phys fci shen ast', 'post phys fci shen ast',
            'pre phys fci shen trn', 'post phys fci shen trn',
            'pre ctrl fci shen ast', 'post ctrl fci shen ast', 
            'pre ctrl fci shen trn', 'post ctrl fci shen trn', 
            'delta phys fci shen trn', 'delta phys fci shen ast',
            'delta ctrl fci shen trn', 'delta ctrl fci shen ast']

In [ ]:
index = pd.MultiIndex.from_product((sessions, iqs))
brainXIQ_df = pd.DataFrame(index=index, columns=['F-statistic', 'Prob (F-statistic)', 
                                                     'AIC', 'BIC', 'IQ_pval', 'IQXClass_pval'])
all_outcome_mc,_ = liji_sidak_mc(all_df[outcomes], 0.05)
for i in np.arange(0,len(outcomes)):
    for iq_meas in iqs:
        y = all_df[outcomes[i]]
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}1'.format(iq_meas), 
                        #'{0}1XSex'.format(iq_meas), 
                        '{0}1XClass'.format(iq_meas),
                        #'{0}1XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
                        
        else:
            x = all_df[['{0}1'.format(iq_meas), 
                        #'{0}1XSex'.format(iq_meas), 
                        '{0}1XClass'.format(iq_meas),
                        #'{0}1XClassXSex'.format(iq_meas)
                      ] 
                        + covariates]
            
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_pre_iq = reg1.fit()
        brainXIQ_df.at[('pre', iq_meas), 'F-statistic'] = brain_on_pre_iq.fvalue
        brainXIQ_df.at[('pre', iq_meas), 'Prob (F-statistic)'] = brain_on_pre_iq.f_pvalue
        brainXIQ_df.at[('pre', iq_meas), 'AIC'] = brain_on_pre_iq.aic
        brainXIQ_df.at[('pre', iq_meas), 'BIC'] = brain_on_pre_iq.bic
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQ_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[1]
        brainXIQ_df.at[('pre', iq_meas), '{0}_IQXClass_pval'.format(iq_meas)] = brain_on_pre_iq.pvalues[2]
        if brain_on_pre_iq.f_pvalue < all_mc_alpha:
            if brain_on_pre_iq.pvalues['{0}1'.format(iq_meas)] < 0.05 or  brain_on_pre_iq.pvalues[2] < 0.05:
                print(brain_on_delta_iq.summary())
        #print('Course grade regressed on pre IQ, female students\n', grade_on_iq.summary())
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['{0}2'.format(iq_meas), 
                        #'{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        #'{0}2XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates]
        else:
            x = all_df[['{0}2'.format(iq_meas), 
                        #'{0}2XSex'.format(iq_meas), 
                        '{0}2XClass'.format(iq_meas),
                        #'{0}2XClassXSex'.format(iq_meas)
                       ] 
                       + covariates]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_post_iq = reg1.fit()
        brainXIQ_df.at[('post', iq_meas), 'F-statistic'] = brain_on_post_iq.fvalue
        brainXIQ_df.at[('post', iq_meas), 'Prob (F-statistic)'] = brain_on_post_iq.f_pvalue
        brainXIQ_df.at[('post', iq_meas), 'AIC'] = brain_on_post_iq.aic
        brainXIQ_df.at[('post', iq_meas), 'BIC'] = brain_on_post_iq.bic
        brainXIQ_df.at[('post', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[1]
        brainXIQ_df.at[('post', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_post_iq.pvalues[2]
        if brain_on_post_iq.f_pvalue < all_mc_alpha:
            if brain_on_post_iq.pvalues['{0}2'.format(iq_meas)] < 0.05 or  brain_on_post_iq.pvalues[2] < 0.05:
                print(brain_on_delta_iq.summary())
        #print('Course grade regressed on post IQ, female students\n', grade_on_iq.summary())
        
        if outcomes[0][:-9][-4:] != 'rest':
            x = all_df[['delta{0}'.format(iq_meas), 
                        #'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas), 
                        #'delta{0}XClassXSex'.format(iq_meas), 
                        '{0} fd'.format(outcomes[i][:-9])] + covariates
                      ]
        else:
            x = all_df[['delta{0}'.format(iq_meas), 
                        #'delta{0}XSex'.format(iq_meas), 
                        'delta{0}XClass'.format(iq_meas),
                        #'delta{0}XClassXSex'.format(iq_meas)
                       ] 
                       + covariates
                      ]
        x = sm.add_constant(x)
        reg1 = sm.OLS(endog=y, 
                      exog=x, 
                      missing='drop')
        brain_on_delta_iq = reg1.fit()
        brainXIQ_df.at[('delta', iq_meas), 'F-statistic'] = brain_on_delta_iq.fvalue
        brainXIQ_df.at[('delta', iq_meas), 'Prob (F-statistic)'] = brain_on_delta_iq.f_pvalue
        brainXIQ_df.at[('delta', iq_meas), 'AIC'] = brain_on_delta_iq.aic
        brainXIQ_df.at[('delta', iq_meas), 'BIC'] = brain_on_delta_iq.bic
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQ_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[1]
        brainXIQ_df.at[('delta', iq_meas), '{0}_IQXClass_pval'.format(outcomes[i])] = brain_on_delta_iq.pvalues[2]
        if brain_on_delta_iq.f_pvalue < all_mc_alpha:
            if brain_on_delta_iq.pvalues['delta{0}'.format(iq_meas)] < 0.05 or  brain_on_delta_iq.pvalues[2] < 0.05:
                print(brain_on_delta_iq.summary())
    brainXIQ_df.to_csv(join(data_dir, 'rescored', 'iqX{0}_all.csv'.format(outcomes[i])))

In [ ]:
brain_on_delta_iq.pvalues[2]

# Significant Results
## Resting state
None.
## Retrieval
### Physics
Post Full Scale IQ & Post Charpath, assortativity: IQ, Sex, Class, and all interactions.<br>
Post Perceptual Reasoning & Post Charpath: IQXSex, IQXSexXClass, SexXClass<br>
Delta Perceptual Reasoning & Post Charpath ($\alpha$ < 0.05): IQXSec, IQXSexXClass, SexXClass.
### General
Delta Perceptual Reasoning & Post Charpath, assortativity: IQXSexXClass.

## Reasoning
### Transitive Inference
Delta Perceptual Reasoning & Pre Charpath, assortativity: Class, SexXClass.<br>
Pre Working Memory & Delta Charpath, assortativity: IQ, Sex, Class, and all interactions.<br>
### Baseline
Pre Verbal Comprehension & Pre Charpath, assortativity (not all $\alpha$ < 0.01): IQ, Sex, Class, and all interactions.<br>
Pre Full Scale IQ & Pre Charpath, assortativity (not all $\alpha$ < 0.01): IQ, IQxSex, Sex.<br>
Pre Full Scale IQ & Post Charpath, assortativity ($\alpha$ < 0.05): IQXSex, IQXClass, IQXSexXClass, Sex, Class (eff only), SexXClass (chl only).<br>

## Force Concept Inventory
### FCI Questions
Post Full Scale IQ & Post assortativity: IQ, Sex, Class, and all the interactions.<br>
Post Full Scale IQ & Post Charpath: IQXSex, IQXClass•, IQXSexXClass, Sex•, SexXClass.<br>
Delta Full Scale IQ & Post assortativity: IQ, IQXSex, IQXClass•, IQXSexXClass.<br>
Post Pereptual Reasoning & Post assortativity: IQXSex•, IQXSexXClass, Sex•, SexXClass•.<br>
Post Perceptual Reasoning & Post Charpath: IQXSex, IQXClass•, IQXSexXClass, Sex*, SexXClass.<br>
Delta Perceptual Reasoning & Post Charpath, assortativity: IQ, IQXSex, IQXClass, IQXSexXClass.<br>
### Baseline Questions
Post Full Scale IQ & Post Charpath, assortativity ($\alpha$ < 0.05): IQXSex, Sex.

# time for figures!

In [ ]:
crayons_l = sns.crayon_palette(['Vivid Tangerine', 'Cornflower'])
crayons_d = sns.crayon_palette(['Brick Red', 'Midnight Blue'])

f = sns.diverging_palette(8.2, 44, s=85, l=50, n=200)
m = sns.diverging_palette(243, 278, s=85, l=50, n=200)

f_2 = sns.crayon_palette(['Red Orange', 'Vivid Tangerine'])
m_2 = sns.crayon_palette(['Cerulean', 'Cornflower'])

# Physics Retrieval & PRIQ

In [ ]:
h = sns.lmplot('PRI2', 'post phys retr shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10., )

h.set_xlabels('Post-instruction perceptual reasoning')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'RETR_phys_post-cplXpost-priq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('PRI2', 'post phys retr shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction perceptual reasoning')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'RETR_phys_post-cplXpost-priq_f.png'), dpi=300)

# General retrieval and deltaPRIQ

In [ ]:
h = sns.lmplot('deltaPRI', 'post gen retr shen mod', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Change in perceptual reasoning')
h.set_ylabels('Post-instruction network transitivity')
h.savefig(join(fig_dir, 'RETR_gen_post-modXdelta-priq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaPRI', 'post gen retr shen mod', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Change in perceptual reasoning')
h.set_ylabels('Post-instruction network transitivity')
h.savefig(join(fig_dir, 'RETR_gen_post-modXdelta-priq_f.png'), dpi=300)

# Physics Retrieval & FSIQ

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys retr shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction network assortativity')
h.savefig(join(fig_dir, 'RETR_phys_post-cplXpost-fsiq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys retr shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction characteristic path length')
#h.savefig(join(fig_dir, 'pre_prXpre_eff_inf-m.png'), dpi=300)

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys retr shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction network-assortativity')
h.savefig(join(fig_dir, 'RETR_phys_post-cplXpost-fsiq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys retr shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction characteristic path length')
#h.savefig(join(fig_dir, 'pre_prXpre_eff_inf-m.png'), dpi=300)

# FCI and PRIQ

In [ ]:
h = sns.lmplot('PRI2', 'post phys fci shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction perceptual reasoning')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'FCI_phys_post-cplXpost-priq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('PRI2', 'post phys fci shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction perceptual reasoning')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'FCI_phys_post-cplXpost-priq_m.png'), dpi=300)

# FCI and FSIQ

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys fci shen cpl', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'FCI_phys_post-cplXpost-fsiq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('FSIQ2', 'post phys fci shen cpl', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Post-instruction full-scale IQ')
h.set_ylabels('Post-instruction characteristic path length')
h.savefig(join(fig_dir, 'FCI_phys_post-cplXpost-fsiq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaFSIQ', 'post phys fci shen eff', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Change in full-scale IQ')
h.set_ylabels('Post-instruction network assortativity')
h.savefig(join(fig_dir, 'FCI_phys_post-effXdelta-fsiq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaPRI', 'post phys fci shen eff', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Change in perceptual reasoning')
h.set_ylabels('Post-instruction network assortativity')
h.savefig(join(fig_dir, 'FCI_phys_post-effXdelta-priq_m.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaFSIQ', 'post phys fci shen eff', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Change in full-scale IQ')
h.set_ylabels('Post-instruction network assortativity')
h.savefig(join(fig_dir, 'FCI_phys_post-effXdelta-fsiq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('deltaPRI', 'post phys fci shen eff', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Change in perceptual reasoning')
h.set_ylabels('Post-instruction network assortativity')
h.savefig(join(fig_dir, 'FCI_phys_post-effXdelta-priq_f.png'), dpi=300)

# Reasoning and WMIQ

In [ ]:
h = sns.lmplot('WMI1', 'delta infr reas shen eff', data=df_f, hue='Mod', 
           palette=f_2, legend_out=True, height=10.)

h.set_xlabels('Pre-instruction working memory')
h.set_ylabels('Change in network assortativity')
h.savefig(join(fig_dir, 'REAS_infr_delta-effXpre-wmiq_f.png'), dpi=300)

In [ ]:
h = sns.lmplot('WMI1', 'delta infr reas shen eff', data=df_m, hue='Mod', 
           palette=m_2, legend_out=True, height=10.)

h.set_xlabels('Pre-instruction working memory')
h.set_ylabels('Change in network assortativity')
h.savefig(join(fig_dir, 'REAS_infr_delta-effXpre-wmiq_m.png'), dpi=300)

In [ ]:
import seaborn as sns
sns.set()